# Vorflow Comprehensive Demo

This notebook demonstrates the full capabilities of the `vorflow` package for generating unstructured Voronoi grids for MODFLOW 6.

## Workflow
1. **Define Geometry**: Create Shapely objects (Polygons, Lines, Points).
2. **ConceptualMesh**: Register these objects with refinement parameters.
3. **MeshGenerator**: Generate a triangular mesh using Gmsh.
4. **VoronoiTessellator**: Convert the triangles to a Voronoi grid.
5. **Analysis**: Inspect mesh quality.

In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
from shapely.geometry import Point, Polygon, LineString, box

from vorflow import ConceptualMesh, MeshGenerator, VoronoiTessellator
from vorflow.utils import calculate_mesh_quality, summarize_quality

## 1. Define Geometry
We define a domain with a river, a fault, a well, and a refined zone.

In [ ]:
# Domain: 200x200 box
domain = box(0, 0, 200, 200)
# add a hole to the domain
hole = box(80, 80, 120, 120)
domain = Polygon(domain.exterior.coords, [hole.exterior.coords])


# Refined Zone: A smaller box inside
zone_poly = box(50, 50, 75, 75)

# River: A sine wave
river_coords = [(x, 75 + 20 * np.sin(0.1 * x)) for x in range(-10, 220, 10)]
river_line = LineString(river_coords)

# Fault: A straight line (Barrier)
fault_line = LineString([(100, 0), (100, 150)])

# Well: A point
well_point1 = Point(25, 25)
well_point2 = Point(60,60)

## 2. Build Conceptual Mesh
Here we configure the mesh refinement.
- **resolution**: Target edge length.
- **dist_min**: Distance from the feature where resolution is constant.
- **dist_max**: Distance where resolution decays to background size.

In [ ]:
cm = ConceptualMesh(crs="EPSG:3857")

# 1. Add Domain (Background)
cm.add_polygon(domain, zone_id=1, resolution=20.0)

# 2. Add Refined Zone
# dist_max_out controls how fast the mesh grows outside this zone
cm.add_polygon(zone_poly,z_order=2,  zone_id=2, resolution=3.0, dist_max_out=20.0)


# 3. Add River (Standard Line)
cm.add_line(river_line, line_id="river", resolution=2.0, dist_max=15.0)

# 4. Add Fault (Barrier)
# is_barrier=True ensures the mesh edges align with this line
cm.add_line(fault_line, line_id="fault", resolution=2.0, is_barrier=True, dist_max=15.0)

# 5. Add Well
cm.add_point(well_point1, point_id="well-a", resolution=1.0, dist_max=50.0)
cm.add_point(well_point2, point_id="well-b", resolution=1.0, dist_max=20.0)

# Process inputs
clean_polys, clean_lines, clean_pts = cm.generate()

## 3. Generate Triangular Mesh
We use Gmsh to generate the underlying triangulation.

In [ ]:
mg = MeshGenerator(background_lc=20.0, verbosity=2)
mg.generate(clean_polys, clean_lines, clean_pts)

## 4. Voronoi Tessellation
Convert the triangular mesh to a Voronoi grid, clipped to the domain.

In [ ]:
vt = VoronoiTessellator(mg, cm, clip_to_boundary=True)
grid = vt.generate()

## 5. Visualization
Plot the resulting grid along with the input features.

In [ ]:
fig, ax = plt.subplots(figsize=(12, 12))
ax.set_aspect('equal')

# Plot Grid
grid.plot(ax=ax, edgecolor='black', linewidth=0.5, alpha=0.5, color='white')

# Plot Features
gpd.GeoSeries([domain]).boundary.plot(ax=ax, color='black', linewidth=2, label='Domain')
gpd.GeoSeries([zone_poly]).boundary.plot(ax=ax, color='blue', linestyle='--', label='Refined Zone')
gpd.GeoSeries([river_line]).plot(ax=ax, color='cyan', linewidth=2, label='River')
gpd.GeoSeries([fault_line]).plot(ax=ax, color='red', linewidth=2, linestyle='-', label='Fault (Barrier)')
gpd.GeoSeries([well_point1]).plot(ax=ax, color='red', marker='*', markersize=100, label='Well')
gpd.GeoSeries([well_point2]).plot(ax=ax, color='red', marker='*', markersize=100,)

plt.legend()
plt.title("Voronoi Grid with Refinement")
plt.show()

## 6. Quality Analysis
Check the geometric quality of the grid cells.

In [ ]:
quality_gdf = calculate_mesh_quality(grid, calc_ortho=True)
summarize_quality(quality_gdf)

# Plot Orthogonality Error
fig, ax = plt.subplots(figsize=(10, 8))
quality_gdf.plot(column='ortho_error', ax=ax, legend=True, cmap='Reds', vmin=0, vmax=10)
plt.title("Orthogonality Error (Degrees)")
plt.show()

In [ ]:
# Plot Orthogonality Error
fig, ax = plt.subplots(figsize=(10, 8))
quality_gdf.plot(column='drift_ratio', ax=ax, legend=True, cmap='Reds', vmin=0, vmax=.4)
plt.title("Drift Ratio")
plt.show()